---

This notebook was created as a project for the course "Introduction to Data Science in Python" from the University of Michigan via Coursera.

---

In [16]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import re
from scipy import stats

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. (`price_ratio=quarter_before_recession/recession_bottom`)


In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [14]:
def get_list_of_university_towns():
    '''Returns a dataframe of university towns and their states'''
    
    uni_towns_imp = pd.read_csv('university_towns.txt', error_bad_lines=False, header=None, names = ['RegionName'],sep='\n')
    #create a field for state
    uni_towns = uni_towns_imp
    uni_towns['State'] = np.NaN
    #identify the states by the presence of string "[edit]" and add them to the State field
    for i in range(0,len(uni_towns)-1):
        if re.search(r'\[edit\]',uni_towns['RegionName'][i]):
            uni_towns.loc[i,'State']=uni_towns['RegionName'][i]
    #forward fill the nulls to populate the proper state for each university town
    uni_towns['State'] = uni_towns['State'].ffill()
    #clean up the extra text, get rid of the state-state rows, set state as index
    uni_towns = uni_towns[['State', 'RegionName']].replace(r'\[edit\]', '', regex=True).replace(r' \(.*', '', regex=True)
    uni_towns = uni_towns[uni_towns.State != uni_towns.RegionName].reset_index()
    del uni_towns['index']

    return uni_towns

In [13]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time'''
    
    gdp = pd.read_excel('gdplev.xls', header=5, parse_cols='E:G').iloc[214:]
    del gdp['GDP in billions of current dollars']
    gdp.columns = ['GDP 09 dlr']

    #find and return the first quarter of the recession. defined by two consecuive quarters of negative GDP growth
    rec_start = ''
    for i in range(2,len(gdp)):
        if ((gdp['GDP 09 dlr'][i] < gdp['GDP 09 dlr'][i-1]) and (gdp['GDP 09 dlr'][i-1] < gdp['GDP 09 dlr'][i-2])):
            rec_start = gdp.index[i-1]
            break

    return str(rec_start)

In [12]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time'''
    
    gdp = pd.read_excel('gdplev.xls', header=5, parse_cols='E:G').iloc[214:]
    del gdp['GDP in billions of current dollars']
    gdp.columns = ['GDP 09 dlr']
    
    rec_start_row = gdp.index.get_loc(get_recession_start())
    #begin looking for the recession end only after one has begun
    for i in range(rec_start_row,len(gdp)):
        if ((gdp['GDP 09 dlr'][i] > gdp['GDP 09 dlr'][i-1]) and (gdp['GDP 09 dlr'][i-1] > gdp['GDP 09 dlr'][i-2])):
            rec_end = gdp.index[i]
            break

    return str(rec_end)

In [11]:
def get_recession_bottom():
        '''Returns the year and quarter of the lowest GDP during the recession'''

        gdp = pd.read_excel('gdplev.xls', header=5, parse_cols='E:G').iloc[214:]
        del gdp['GDP in billions of current dollars']
        gdp.columns = ['GDP 09 dlr']

        rec_start_row = gdp.index.get_loc(get_recession_start())
        rec_end_row = gdp.index.get_loc(get_recession_end())
    
        return str(gdp.iloc[rec_start_row:rec_end_row+1]['GDP 09 dlr'].idxmin())

In [10]:
def convert_housing_data_to_quarters():
        '''Converts the housing data from monthly to quarterly using the mean price'''
        
        z_housing = pd.read_csv('City_Zhvi_AllHomes.csv',index_col=[2])
        #convert the states dictionary to a dataframe
        states1 = pd.DataFrame(list(states.items()), columns=['abbr', 'State'])
        #replace the state abbreviations with full state names in the housing dataframe
        housing = pd.merge(z_housing, states1, how='left', left_index=True, right_on='abbr').set_index(['State', 'RegionName'])
        del housing['abbr']
        #use only data starting with January 2000
        c = housing.columns.tolist()
        cols = c[49:]
        housing = housing[cols].sort_index()
        #use pandas PeriodIndex to convert monthly data to quarterly using the mean price
        housing = housing.groupby(pd.PeriodIndex(housing.columns, freq='q'), axis=1).mean()
        cols1 = []
        for x in housing.columns: cols1.append(str(x).lower())
        housing.columns = cols1

        return housing

In [18]:

def run_ttest():
    '''Test null hypothesis: the mean price ratio (quarter before the recession start / bottom quarter of recession)
    of university towns is not significantly different from non-university towns'''
    
    housing = convert_housing_data_to_quarters()
    #make a new dataframe for the price ratio
    a = housing.columns.get_loc(get_recession_start()) - 1
    b = housing.columns.get_loc(get_recession_bottom())
    housing['price_ratio'] = housing[housing.columns[a]]/housing[housing.columns[b]]
    housing100 = pd.DataFrame(housing['price_ratio'])

    uni_towns1 = get_list_of_university_towns()

    #get a dataframe of price ratios for university towns
    housing_uni = pd.merge(housing100, uni_towns1, left_index=True, right_on=['State','RegionName']).set_index(['State', 'RegionName'])
    #get a dataframe of price ratios for non-university towns
    index_to_keep = housing100.index.symmetric_difference(housing_uni.index)
    housing_non = housing100.loc[index_to_keep]

    #perform an independent sample t-test
    test1 = stats.ttest_ind(housing_uni['price_ratio'], housing_non['price_ratio'], equal_var=False, nan_policy='omit')

    accrej = 'fail to reject'
    lowhigh = 'less'

    if test1.pvalue < 0.01:
        accrej = 'reject'
        if housing_uni['price_ratio'].mean() > housing_non['price_ratio'].mean():
            lowhigh = 'greater'
    else: lowhigh = 'neither greater nor less'

    result = 'With a p-value of ' + str(test1.pvalue) + ', we ' + accrej + ' the null hypothesis at the significance level of 0.01.'
    result2 = 'The decline in housing prices in university towns was ' + lowhigh + ' than that of non-university towns during the recession in 2008-2009.'
    result3 = 'The mean university town ratio of pre-recession to recession-bottom housing prices is ' + str(round(housing_uni['price_ratio'].mean(),4)) + '.'
    result4 = 'That of non-university towns is ' + str(round(housing_non['price_ratio'].mean(),4)) + '.'


    return result + '\n' + result2 + '\n' + result3 + '\n' + result4

print(run_ttest())

With a p-value of 7.74416917172e-05, we reject the null hypothesis at the significance level of 0.01.
The decline in housing prices in university towns was less than that of non-university towns during the recession in 2008-2009.
The mean university town ratio of pre-recession to recession-bottom housing prices is 1.0614.
That of non-university towns is 1.0794.
